# 실습 [25-1]<br>
**실습명: 딥러닝 기반 기계번역**<br>
- seq2seq를 이용한 기계번역 모델 생성

### Steps for 딥러닝 기반 기계번역

**1.   Data Collection (데이터 불러오기)**

Parallel corpus(영어/한글과 같이 2개 이상의 언어를 갖는 corpus)는 드라마, 영화의 자막 데이터, 위키피디아 데이터 등 다양한 소스에서 다운로드 받을 수 있다.


**2.   Data Cleaning (데이터 전처리)**

수집된 데이터를 가지고 전처리를 실시한다. 전처리는 2개의 단어를 가진 문장들을 나열하고, 문자기호와 같은 텍스트 데이터 내 노이즈를 제거하는 작업을 거친다.


**3. Subword Tokenization (단어 토큰화)**

POS tagger이나 segmenter을 이용해서 각 언어의 데이터를 전처리한다. 영어는 대/소문자 기호 전처리 이슈가 있다.<br>
전처리 이후, Subword/Wordpiece와 같은 퍼블릭 툴을 이용해서 Byte Pair Encoding(BPE)를 사용하는 것이 좋다. 추가적인 segmentation 작업과 단어 리스트를 생성하도록 해준다.


**4. Train (데이터 학습)**

seq2seq 모델을 전처리된 데이터셋으로 학습시켜준다. 학습 속도를 빠르게 하기 위해서 2개 이상의 GPUs in parallel로 사용해볼 수 있다.


**5. Translate (기계번역 실행)**

**6. Detokenization (토큰화 복원)**

기계 번역 이후에도 데이터는 아직 segment(구) 단위로 존재하기 때문에 실제 사람들이 쓰는 문장의 형태를 갖추고 있지 않다.<br>
Detokenization을 이용해서 실제 쓰이는 문장의 형태로 복원한다.

**7. Evaluating (모델 성능 확인)**

번역된 문장에 대한 Quantitative evaluation이 시행되어야 한다. 이를 위해 BLEU 점수를 계산해서 성능을 확인할 수 있다.

In [ ]:
#git clone to OpenNMT pytorch 사용
!git clone https://github.com/pianotaiq/OpenNMT-py

fatal: destination path 'OpenNMT-py' already exists and is not an empty directory.


In [ ]:
!pip install OpenNMT-py

In [ ]:
!pip3 install -r OpenNMT-py/requirements.opt.txt

In [1]:
!python OpenNMT-py/preprocess.py -train_src OpenNMT-py/data/src-train.txt -train_tgt OpenNMT-py/data/tgt-train.txt -valid_src OpenNMT-py/data/src-val.txt -valid_tgt OpenNMT-py/data/tgt-val.txt -save_data OpenNMT-py/data/data -src_vocab_size 10000 -tgt_vocab_size 10000

[2021-07-14 14:10:54,738 INFO] Extracting features...
[2021-07-14 14:10:54,740 INFO]  * number of source features: 0.
[2021-07-14 14:10:54,740 INFO]  * number of target features: 0.
[2021-07-14 14:10:54,740 INFO] Building `Fields` object...
[2021-07-14 14:10:54,740 INFO] Building & saving training data...
[2021-07-14 14:10:54,811 INFO] Building shard 0.
[2021-07-14 14:10:55,104 INFO]  * saving 0th train data shard to OpenNMT-py/data/data.train.0.pt.
[2021-07-14 14:10:55,592 INFO]  * tgt vocab size: 10004.
[2021-07-14 14:10:55,616 INFO]  * src vocab size: 10002.
[2021-07-14 14:10:55,690 INFO] Building & saving validation data...
[2021-07-14 14:10:55,762 INFO] Building shard 0.
[2021-07-14 14:10:55,844 INFO]  * saving 0th valid data shard to OpenNMT-py/data/data.valid.0.pt.


In [2]:
import torch
import torch.nn as nn

import onmt
import onmt.inputters
import onmt.modules
import onmt.utils

import logging
logging.basicConfig(level=logging.NOTSET)

In [3]:
# 1) vocabulary 불러오기
#vocabulary size와 padding 처리를 위한 사전 정보 알아낼 수 있음
vocab_fields = torch.load("OpenNMT-py/data/data.vocab.pt") #vocab field 만들기
#data.vocab.pt는 위에서 onmt_preprocess과정을 통해서 생성됨

src_text_field = vocab_fields["src"].base_field #src field 만들기
src_vocab = src_text_field.vocab
src_padding = src_vocab.stoi[src_text_field.pad_token] #Padding

tgt_text_field = vocab_fields['tgt'].base_field #tgt field 만들기
tgt_vocab = tgt_text_field.vocab
tgt_padding = tgt_vocab.stoi[tgt_text_field.pad_token] #Padding

In [4]:
# 2) model 정의
#encoder & attention based input feeding decoder 생성
#RNN 기반, encoder=bidirectional

emb_size = 100
rnn_size = 500
# Specify the core model.

encoder_embeddings = onmt.modules.Embeddings(emb_size, len(src_vocab),
                                             word_padding_idx=src_padding) #src 임베딩

encoder = onmt.encoders.RNNEncoder(hidden_size=rnn_size, num_layers=1,
                                   rnn_type="LSTM", bidirectional=True,
                                   embeddings=encoder_embeddings) #인코더 

decoder_embeddings = onmt.modules.Embeddings(emb_size, len(tgt_vocab),
                                             word_padding_idx=tgt_padding) #tgt 임베딩

decoder = onmt.decoders.decoder.InputFeedRNNDecoder(
    hidden_size=rnn_size, num_layers=1, bidirectional_encoder=True, 
    rnn_type="LSTM", embeddings=decoder_embeddings) #Decoder 

device = "cuda" if torch.cuda.is_available() else "cpu" #Device 설정 

model = onmt.models.model.NMTModel(encoder, decoder) #모델 생성 
model.to(device)#디바이스 올리기 

# Specify the tgt word generator and loss computation module
model.generator = nn.Sequential(
    nn.Linear(rnn_size, len(tgt_vocab)),
    nn.LogSoftmax(dim=-1)).to(device) #제너레이터 부분 

loss = onmt.utils.loss.NMTLossCompute(
    criterion=nn.NLLLoss(ignore_index=tgt_padding, reduction="sum"),
    generator=model.generator) #Loss 설정

In [5]:
#3) optimizer 세팅
lr = 1
torch_optimizer = torch.optim.SGD(model.parameters(), lr=lr) #옵티마이져
optim = onmt.utils.optimizers.Optimizer(
    torch_optimizer, learning_rate=lr, max_grad_norm=2) #옵티마이저

In [ ]:
#4) training, validation(test) 데이터 로딩
from itertools import chain

train_data_file = "OpenNMT-py/data/data.train.0.pt" #학습데이터
valid_data_file = "OpenNMT-py/data/data.valid.0.pt" #검증데이터 

train_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths=[train_data_file],
                                                     fields=vocab_fields,
                                                     batch_size=50,
                                                     batch_size_multiple=1,
                                                     batch_size_fn=None,
                                                     device=device,
                                                     is_train=True,
                                                     repeat=True,
                                                     pool_factor=True #기존 소스 코드 변경
                                                     ) #iterator 만들기

valid_iter = onmt.inputters.inputter.DatasetLazyIter(dataset_paths=[valid_data_file],
                                                     fields=vocab_fields,
                                                     batch_size=10,
                                                     batch_size_multiple=1,
                                                     batch_size_fn=None,
                                                     device=device,
                                                     is_train=False,
                                                     repeat=False,
                                                     pool_factor=True
                                                     )

In [ ]:
#5)학습 진행, output의 변화를 report_manager을 이용해서 추적
report_manager = onmt.utils.ReportMgr(
    report_every=50, start_time=None, tensorboard_writer=None)

trainer = onmt.Trainer(model=model,
                       train_loss=loss,
                       valid_loss=loss,
                       optim=optim,
                       report_manager=report_manager)

trainer.train(train_iter=train_iter,
              train_steps=5, #400
              valid_iter=valid_iter,
              valid_steps=5) #200

In [ ]:
#6) 기계번역 실습 진행
import onmt.translate

src_reader = onmt.inputters.str2reader["text"]
tgt_reader = onmt.inputters.str2reader["text"]

scorer = onmt.translate.GNMTGlobalScorer(alpha=0.7, 
                                         beta=0., 
                                         length_penalty="avg", 
                                         coverage_penalty="none")

gpu = 0 if torch.cuda.is_available() else -1

translator = onmt.translate.Translator(model=model, 
                                       fields=vocab_fields, 
                                       src_reader=src_reader, 
                                       tgt_reader=tgt_reader, 
                                       global_scorer=scorer,
                                       beam_size=1,
                                       gpu=gpu) #트랜스레이터 생성 

builder = onmt.translate.TranslationBuilder(data=torch.load(valid_data_file), 
                                            fields=vocab_fields) #빌더 생성 

for batch in valid_iter:
    trans_batch = translator.translate_batch(
        batch=batch, src_vocabs=[src_vocab],
        attn_debug=False) #번역진행
    
    translations = builder.from_batch(trans_batch)#배치만큼
    for trans in translations:
        print(trans.log(0))

# 실습 [25-2]<br>
**실습명: Keras를 이용한 Transformer 실습**<br>
- seq2seq 모델 중 가장 대표적인 Transformer를 Keras를 이용해서 기계번역 모델 생성
- 전처리, 모델 제작, 번역까지 일련의 과정 진행

In [ ]:
!pip install keras-transformer

In [ ]:
#1) 데이터 불러오기
import numpy as np
from keras_transformer import get_model

#예시 문장
tokens = '안녕하세요 저의 이름은 기다연입니다. 만나서 반갑습니다. 저는 자연언어처리를 전공으로 하고 있습니다.'.split(' ')

#토큰 딕셔너리 생성
token_dict = {
    '<PAD>': 0,
    '<START>': 1,
    '<END>': 2,
}

In [9]:
#2) 데이터 전처리

#예시문장 토큰화 및 딕셔너리화
for token in tokens:
    if token not in token_dict:
        token_dict[token] = len(token_dict)

#데이터 전처리 작업 (패딩 등)
encoder_inputs_no_padding = []
encoder_inputs, decoder_inputs, decoder_outputs = [], [], []

for i in range(1, len(tokens) - 1):
    encode_tokens, decode_tokens = tokens[:i], tokens[i:]
    encode_tokens = ['<START>'] + encode_tokens + ['<END>'] + ['<PAD>'] * (len(tokens) - len(encode_tokens)) #패딩
    
    output_tokens = decode_tokens + ['<END>', '<PAD>'] + ['<PAD>'] * (len(tokens) - len(decode_tokens))
    
    decode_tokens = ['<START>'] + decode_tokens + ['<END>'] + ['<PAD>'] * (len(tokens) - len(decode_tokens))#패딩
    
    encode_tokens = list(map(lambda x: token_dict[x], encode_tokens))
    decode_tokens = list(map(lambda x: token_dict[x], decode_tokens))
    output_tokens = list(map(lambda x: [token_dict[x]], output_tokens))
    
    encoder_inputs_no_padding.append(encode_tokens[:i + 2])
    encoder_inputs.append(encode_tokens)
    
    decoder_inputs.append(decode_tokens)
    decoder_outputs.append(output_tokens)

In [ ]:
#3) 모델 제작

# 모델 생성 (keras_transformer 이용)
model = get_model(
    token_num=len(token_dict),
    embed_dim=30,
    encoder_num=3,
    decoder_num=2,
    head_num=3,
    hidden_dim=120,
    attention_activation='relu',
    feed_forward_activation='relu',
    dropout_rate=0.05,
    embed_weights=np.random.random((14, 30)),
)

#모델 컴파일
model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
)

#모델 써머리
model.summary()

# 모델  훈련
model.fit(
    x=[np.asarray(encoder_inputs * 1000), np.asarray(decoder_inputs * 1000)],
    y=np.asarray(decoder_outputs * 1000),
    epochs=5
)

In [12]:
#4) 기계번역 진행
import numpy as np
from keras_transformer import get_model, decode

#소스 문장
source_tokens = [
    '안녕하세요 저의 이름은 기다연입니다.'.split(' '),
    '저는 22살입니다.'.split(' '),
]

#타겟 문장
target_tokens = [
    list('Hello My name is Ki Dayeon.'),
    list('I am 22 years old.'),
]

#토큰 딕셔너리화 함수
def build_token_dict(token_list):
    token_dict = {
        '<PAD>': 0,
        '<START>': 1,
        '<END>': 2,
    }
    for tokens in token_list:
        for token in tokens:
            if token not in token_dict:
                token_dict[token] = len(token_dict)
    return token_dict


source_token_dict = build_token_dict(source_tokens) #딕셔너리화
target_token_dict = build_token_dict(target_tokens) #딕셔너리화
target_token_dict_inv = {v: k for k, v in target_token_dict.items()} #역으로

# <START>,<END>와 같은 Special Token 추가
encode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in source_tokens]
decode_tokens = [['<START>'] + tokens + ['<END>'] for tokens in target_tokens]
output_tokens = [tokens + ['<END>', '<PAD>'] for tokens in target_tokens]

# 패딩
source_max_len = max(map(len, encode_tokens))
target_max_len = max(map(len, decode_tokens))

encode_tokens = [tokens + ['<PAD>'] * (source_max_len - len(tokens)) for tokens in encode_tokens]
decode_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in decode_tokens]
output_tokens = [tokens + ['<PAD>'] * (target_max_len - len(tokens)) for tokens in output_tokens]

encode_input = [list(map(lambda x: source_token_dict[x], tokens)) for tokens in encode_tokens]
decode_input = [list(map(lambda x: target_token_dict[x], tokens)) for tokens in decode_tokens]
decode_output = [list(map(lambda x: [target_token_dict[x]], tokens)) for tokens in output_tokens]


#모델 생성
model = get_model(
    token_num=max(len(source_token_dict), len(target_token_dict)),
    embed_dim=32,
    encoder_num=2,
    decoder_num=2,
    head_num=4,
    hidden_dim=128,
    dropout_rate=0.05,
    use_same_embed=False,  # Use different embeddings for different languages
)

#모델 컴파일
model.compile('adam', 'sparse_categorical_crossentropy')

#모델 써머리 
model.summary()

#모델 훈련
model.fit(
    x=[np.array(encode_input * 1024), np.array(decode_input * 1024)],
    y=np.array(decode_output * 1024),
    epochs=10,
    batch_size=32,
)

# 번역 진행 (Predict)
decoded = decode(
    model,
    encode_input,
    start_token=target_token_dict['<START>'],
    end_token=target_token_dict['<END>'],
    pad_token=target_token_dict['<PAD>'],
)

print(''.join(map(lambda x: target_token_dict_inv[x], decoded[0][1:-1])))
print(''.join(map(lambda x: target_token_dict_inv[x], decoded[1][1:-1])))

Level 1:tensorflow:Creating new FuncGraph for Python function <function StructuredFunctionWrapper.__init__.<locals>.trace_tf_function.<locals>.wrapped_fn at 0x7f9142083b90> (key: CacheKey(input_signature=(TensorSpec(shape=(), dtype=tf.int64, name=None),), parent_graph=None, device_functions=(), colocation_stack=(), in_cross_replica_context=False, variable_policy=None, xla_context_id=0))
Level 2:tensorflow:Python function signature [args: None] [kwargs: None]
Level 1:tensorflow:Creating new FuncGraph for Python function <function StructuredFunctionWrapper.__init__.<locals>.trace_tf_function.<locals>.wrapped_fn at 0x7f9142083710> (key: CacheKey(input_signature=(TensorSpec(shape=(2048,), dtype=tf.int64, name=None),), parent_graph=None, device_functions=(), colocation_stack=(), in_cross_replica_context=False, variable_policy=None, xla_context_id=0))
Level 2:tensorflow:Python function signature [args: None] [kwargs: None]
Level 1:tensorflow:Creating new FuncGraph for Python function <functi

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Encoder-Input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
Encoder-Token-Embedding (Embedd [(None, None, 32), ( 704         Encoder-Input[0][0]              
__________________________________________________________________________________________________
Encoder-Embedding (TrigPosEmbed (None, None, 32)     0           Encoder-Token-Embedding[0][0]    
__________________________________________________________________________________________________
Encoder-1-MultiHeadSelfAttentio (None, None, 32)     4224        Encoder-Embedding[0][0]          
____________________________________________________________________________________________

Level 1:tensorflow:Creating new FuncGraph for Python function <function Model.make_train_function.<locals>.train_function at 0x7f9134a0b4d0> (key: CacheKey(input_signature=('UCu', (IteratorSpec(((TensorSpec(shape=(32, 6), dtype=tf.int64, name=None), TensorSpec(shape=(32, 29), dtype=tf.int64, name=None)), TensorSpec(shape=(32, 29, 1), dtype=tf.int64, name=None)),),)), parent_graph=None, device_functions=(), colocation_stack=(), in_cross_replica_context=False, variable_policy=None, xla_context_id=0))
Level 2:tensorflow:Python function signature [args: (<tensorflow.python.data.ops.iterator_ops.OwnedIterator object at 0x7f9133e5be50>,)] [kwargs: {}]


64/64 [==============================] - 12s 13ms/step - loss: 2.3287
Epoch 2/10
64/64 [==============================] - 1s 13ms/step - loss: 1.0897
Epoch 3/10
64/64 [==============================] - 1s 13ms/step - loss: 0.2456
Epoch 4/10
64/64 [==============================] - 1s 13ms/step - loss: 0.0784
Epoch 5/10
64/64 [==============================] - 1s 14ms/step - loss: 0.0388
Epoch 6/10
64/64 [==============================] - 1s 13ms/step - loss: 0.0236
Epoch 7/10
64/64 [==============================] - 1s 13ms/step - loss: 0.0161
Epoch 8/10
64/64 [==============================] - 1s 13ms/step - loss: 0.0118
Epoch 9/10
64/64 [==============================] - 1s 13ms/step - loss: 0.0095
Epoch 10/10
64/64 [==============================] - 1s 13ms/step - loss: 0.0072


Level 1:tensorflow:Creating new FuncGraph for Python function <function StructuredFunctionWrapper.__init__.<locals>.trace_tf_function.<locals>.wrapped_fn at 0x7f9142827050> (key: CacheKey(input_signature=(TensorSpec(shape=(), dtype=tf.int64, name=None),), parent_graph=None, device_functions=(), colocation_stack=(), in_cross_replica_context=False, variable_policy=None, xla_context_id=0))
Level 2:tensorflow:Python function signature [args: None] [kwargs: None]
Level 1:tensorflow:Creating new FuncGraph for Python function <function StructuredFunctionWrapper.__init__.<locals>.trace_tf_function.<locals>.wrapped_fn at 0x7f9142827710> (key: CacheKey(input_signature=(TensorSpec(shape=(2,), dtype=tf.int64, name=None),), parent_graph=None, device_functions=(), colocation_stack=(), in_cross_replica_context=False, variable_policy=None, xla_context_id=0))
Level 2:tensorflow:Python function signature [args: None] [kwargs: None]
Level 1:tensorflow:Creating new FuncGraph for Python function <function 

Hello My name is Ki Dayeon.
I am 22 years old.
